In [1]:
import uproot
#from encode import encode
import numpy as np
import time
#import numba
import matplotlib.pyplot as plt

In [2]:
from encode import encode, decode

In [3]:
def getAlgoBlockOutputBestChoice(df, nTC, sumEncoding=(0,4,4)):
    SUM = encode(df.calibCharge.sum().astype(np.int),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    TOPNCHARGE = (df.nlargest(nTC,'calibCharge'))
    selidx = TOPNCHARGE.index
    noselidx = list(set(df.index)^set(TOPNCHARGE.index))
    df = df.copy(deep=True)
    df.iloc[noselidx, df.columns.get_loc('calibCharge')] = 0
    ADD_MAP = (df.calibCharge>0).values.astype(np.int)
#    CHARGES = [x for x in df.calibCharge if x != 0]  ### if charges not encoded
    CHARGES = [encode(x,3, 4, 3,asInt=True) for x in df.calibCharge if x != 0]  ### if charges encoded
#    print (ADD_MAP,CHARGES)
    return (ADD_MAP,CHARGES,SUM)


In [4]:
def formatBestChoiceOutput(BC, NTCQ, ADD_MAP, CHARGEQ,SUM):
    header = format(BC, '#0%ib'%(15))[-5:]
    dataType = ''
#    if NTCQ==0: 
#        dataType='00'
#    elif NTCQ<8:
#        dataType='01'
#    else:
#        dataType='10'
    modSumData = format(SUM, '#0%ib'%(10))[2:]
#    extraBit='1'
#    if NTCQ==0:
#        nChannelData=''
#        AddressMapData=''
#        ChargeData=''
    if NTCQ<8:
#        nChannelData=format(NTCQ, '#0%ib'%(3+2))[2:]
#        
        bitmap = np.array([int(x) for x in format(ADD_MAP, '#0%ib'%(48+2))[2:]][::-1])
        channelNumbers = np.arange(48)[bitmap==1]
        channelNumbersBin = [format(x,'#0%ib'%(6+2))[2:] for x in channelNumbers]
        AddressMapData = ''
        for x in channelNumbersBin: AddressMapData += x
#       
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]
#        
    else:
#        nChannelData=''
        AddressMapData=format(ADD_MAP, '#0%ib'%(48+2))[2:]
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]

#    formattedData = header + dataType + modSumData + extraBit + nChannelData + AddressMapData + ChargeData
    formattedData = header + modSumData +  AddressMapData + ChargeData
#    print ("formattedData",len(formattedData))
    if len(formattedData)%16==0:
        nPadBits=0
        paddedData = formattedData
    else:
        nPadBits = 16 - (len(formattedData)%16)
#        print ("nPadBits",nPadBits)
        paddedData = formattedData + '0'*nPadBits

#    print(NTCQ,nPadBits)
#    print (len(header),len(modSumData),len(AddressMapData),len(ChargeData))
 #   print (len(paddedData))
    return paddedData

In [5]:
def getAlgoBlockOutput(df, sumEncoding=(0,4,4)):
    MOD_SUM_0 = encode(df.loc[0:15].calibCharge.sum().astype(np.int),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    MOD_SUM_1 = encode(df.loc[16:31].calibCharge.sum().astype(np.int),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    MOD_SUM_2 = encode(df.loc[32:47].calibCharge.sum().astype(np.int),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    SUM = encode(df.calibCharge.sum().astype(np.int),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)

    NTCQ = df.pass_135.sum()
    ADD_MAP = df.pass_135.values.astype(np.int)
    CHARGEQ = df[df.pass_135].calibCharge_Encoded.values.astype(np.int)
    return (NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2)



def bitmapArrayToInt(bitmap):
    total = 0
    for i in range(len(bitmap)):
        total += bitmap[i]<<i
    return total, format(total, '#0%ib'%(len(bitmap)+2))[2:]

In [6]:
def formatThresholdOutput(BC, NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2):
    header = format(BC, '#0%ib'%(15))[-5:]
    dataType = ''
    if NTCQ==0: 
        dataType='00'
    elif NTCQ<8:
        dataType='01'
    else:
        dataType='10'
    modSumData = format(SUM, '#0%ib'%(10))[2:]
    extraBit='1'
    if NTCQ==0:
        nChannelData=''
        AddressMapData=''
        ChargeData=''
    elif NTCQ<8:
        nChannelData=format(NTCQ, '#0%ib'%(3+2))[2:]
        
        bitmap = np.array([int(x) for x in format(ADD_MAP, '#0%ib'%(48+2))[2:]][::-1])
        channelNumbers = np.arange(48)[bitmap==1]
        channelNumbersBin = [format(x,'#0%ib'%(6+2))[2:] for x in channelNumbers]
        AddressMapData = ''
        for x in channelNumbersBin: AddressMapData += x
        
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]
        
    else:
        nChannelData=''
        AddressMapData=format(ADD_MAP, '#0%ib'%(48+2))[2:]
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]

    formattedData = header + dataType + modSumData + extraBit + nChannelData + AddressMapData + ChargeData
    if len(formattedData)%16==0:
        nPadBits=0
        paddedData = formattedData
    else:
        nPadBits = 16 - (len(formattedData)%16)
#        print (nPadBits)
        paddedData = formattedData + '0'*nPadBits

#    print(NTCQ,nPadBits)
    
    return paddedData

In [7]:
def calibRegToCorr(register):
    ### bits
    ### 5,    4,    3,    2,    1,    0
    ### 1,  1/2,  1/4,  1/8, 1/16, 1/32

    corr = 0.
    corr += 1./32 * (register>>0&1)
    corr += 1./16 * (register>>1&1)
    corr += 1./8  * (register>>2&1)
    corr += 1./4  * (register>>3&1)
    corr += 1./2  * (register>>4&1)
    corr +=  1.   * (register>>5&1)
#    corr = 1. + corr
    return corr
calibRegToCorr = np.vectorize(calibRegToCorr)

calibRegisters = np.ones(48).astype(np.int)
calibRegisters = np.random.choice([28,29,30,31,32,33,34,35,36,37],48)

corrFactor= calibRegToCorr(calibRegisters)
    

In [8]:
####Selection of a single wafer
subdet=3
layer=5
wafer = 31

In [9]:
fName='root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/NewNtuples/ntuple_hgcalNtuples_vbf_hmm_200PU_1.root'
_tree = uproot.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))['hgcalTriggerNtuplizer/HGCalTriggerNtuple']

In [10]:
#load dataframe
#df = _tree.pandas.df( ['tc_subdet','tc_zside','tc_layer','tc_wafer','tc_cell','tc_uncompressedCharge','tc_compressedCharge','tc_data','tc_mipPt'],entrystart=0,entrystop=25)
df = _tree.pandas.df( ['tc_subdet','tc_zside','tc_layer','tc_wafer','tc_cell','tc_uncompressedCharge','tc_compressedCharge','tc_data','tc_mipPt'])
df.columns = ['subdet','zside','layer','wafer','triggercell','uncompressedCharge','compressedCharge','data','mipPt']

#remove unwanted layers
df = df[(df.subdet==subdet) & (df.layer==layer) & (df.wafer==wafer)]

In [11]:
#set index
df.set_index(['subdet','zside','layer','wafer','triggercell'],append=True,inplace=True)
df.reset_index('subentry',drop=True,inplace=True)
df.sort_index(inplace=True)

In [12]:
#split +/- zside into separate entries
df.reset_index(inplace=True)
maxN = df.entry.max()
df.set_index(['zside'],inplace=True)

df.loc[-1,['entry']] = df.loc[-1,['entry']] + maxN+1
df.reset_index(inplace=True)
df.set_index(['entry','subdet','zside','layer','wafer','triggercell'],inplace=True)


In [13]:
#get geometry information
geomName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/triggerGeomV9.root"
geomTree = uproot.open(geomName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["hgcaltriggergeomtester/TreeTriggerCells"]


In [14]:
geomDF = geomTree.pandas.df(['subdet','zside','layer','wafer','triggercell','x','y','z','c_n'])
geomDF['r'] = (geomDF['x']**2 + geomDF['y']**2)**.5
geomDF['eta'] = np.arcsinh(geomDF.z/geomDF.r)
geomDF.set_index(['subdet','zside','layer','wafer','triggercell'],inplace=True)
geomDF['isHDM'] = geomDF.c_n>4
geomDF.sort_index(inplace=True)
geomDF.drop(['x','y','c_n'],axis=1,inplace=True)

In [15]:
#### Need to update layer list in geomdf for subdet 4 and 5 to match df
geomDF.reset_index(inplace=True)
geomDF.loc[geomDF.subdet==4,'layer'] += 28
geomDF.loc[geomDF.subdet==5,'layer'] += 28
geomDF.set_index(['subdet','zside','layer','wafer','triggercell'],inplace=True)


In [16]:
threshold_mipPt = 1.35
fCtoADC = 100./1024.
geomDF['threshold_fC'] = threshold_mipPt*np.cosh(geomDF.eta) *3.43
geomDF['threshold_ADC'] = np.round(geomDF.threshold_fC/fCtoADC).astype(np.int)
#geomDF['threshold_ADC'] = geomDF.threshold_fC/fCtoADC


In [17]:
df.reset_index('entry',inplace=True)

df['isHDM'] = geomDF['isHDM']
# df['eta'] = geomDF['eta']
df['threshold_ADC'] = geomDF['threshold_ADC']
#df['threshold_ADC_int'] = geomDF['threshold_ADC_int']

df.reset_index(inplace=True)
df.set_index(['entry','subdet','zside','layer','wafer','triggercell'],inplace=True)

In [18]:
nExp = 4
nMant = 3
nDropHDM = 3
nDropLDM = 1
roundBits = True

df['encodedCharge'] = np.where(df.isHDM,df.uncompressedCharge.apply(encode,args=(nDropHDM,nExp,nMant,roundBits,True)),df.uncompressedCharge.apply(encode,args=(nDropLDM,nExp,nMant,roundBits,True)))
df['decodedCharge'] = np.where(df.isHDM,df.encodedCharge.apply(decode,args=(nDropHDM,nExp,nMant)),df.encodedCharge.apply(decode,args=(nDropLDM,nExp,nMant)))
df['pass_135'] = df.decodedCharge>df.threshold_ADC

In [19]:
df.reset_index('zside',drop=True,inplace=True)

In [20]:
singleEvent = geomDF.reset_index()
singleEvent = singleEvent[(singleEvent.subdet==subdet) & (singleEvent.zside==1) & (singleEvent.layer==layer) & (singleEvent.wafer==wafer)]
#geomDF = geomDF[(geomDF.subdet==subdet) & (geomDF.zside==1) & (geomDF.layer==layer) & (geomDF.wafer==wafer)]
singleEvent.set_index(['subdet','layer','wafer','triggercell'],inplace=True)

In [24]:

calibInputLines = "BC, "

algoInputLines = "BC, "

algoOutputLines = ""
algoOutputLines += "BC, NTCQ, ADD_MAP, "

for i in range(48):
    calibInputLines += "SM_%i, "%i
    algoInputLines += "CALQ_%i, "%i
    algoOutputLines += "CHARGEQ_%i, "%i


calibInputLines = calibInputLines[:-2]+"\n"
algoInputLines = algoInputLines[:-2]+"\n"
algoOutputLines+="BC, SUM, MOD_SUM_0, MOD_SUM_1, MOD_SUM_2\n"

algoOutputLinesBestChoice = "BC, "



#print (algoOutputLinesBestChoice)

formattedDataLines = 'BC, DATA\n'
formattedDataLinesBestChoice = 'BC, DATA\n'

eventList = df.reset_index().entry.unique()
eventTracker = []


nLayer_nLink = {1  : 2,
                  2  : 2,
                  3  : 2,
                  4  : 2,
                  5  : 2,
                  6  : 2,
                  7  : 4,
                  8  : 2,
                  9  : 5,
                  10 : 2,
                  11 : 4,
                  12 : 2,
                  13 : 3,
                  14 : 2,
                  15 : 2,
                  16 : 2,
                  17 : 2,
                  18 : 2,
                  19 : 2,
                  20 : 2,
                  21 : 2,
                  22 : 2,
                  23 : 2,
                  24 : 2,
                  25 : 2,
                  26 : 2,
                  27 : 2,
                  28 : 2,
                  29 : 2,
                  30 : 2,
                  31 : 2,
                  32 : 2,
                  33 : 2,
                  34 : 2,
                  35 : 2,
                  36 : 2,
                  37 : 2,
                  38 : 2,
                  39 : 2,
                  40 : 2,
                  41 : 2,
                  42 : 2,
                  43 : 2,
                  44 : 2,
                  45 : 2,
                  46 : 2,
                  47 : 2,
                  48 : 2,
                  49 : 2,
                  50 : 2,
                  51 : 2,
                  52 : 2}
    
nLink_nTC = {0:0, 1:1, 2:3, 3:6, 4:9, 5:14, 6:18, 7:23, 8:27, 9:32, 10:37, 11:41, 12:46}
nTC_bestchoice = nLink_nTC[nLayer_nLink[layer]]
for i in range(nTC_bestchoice):
    algoOutputLinesBestChoice += "CHARGE_%i, "%i
algoOutputLinesBestChoice += "ADD_MAP, SUM\n"



for BC in range(100):
#for BC in range(1):
    event = np.random.choice(eventList)
    eventTracker.append(event)

    singleEvent['encodedCharge'] = df.loc[event]['encodedCharge']
    singleEvent['decodedCharge'] = df.loc[event]['decodedCharge']
    singleEvent.encodedCharge.fillna(0,inplace=True)
    singleEvent.decodedCharge.fillna(0,inplace=True)

    singleEvent['calibCharge'] = (singleEvent.decodedCharge * corrFactor).round().astype(np.int)
    singleEvent['calibCharge_Encoded'] = np.where(singleEvent.isHDM,singleEvent.calibCharge.apply(encode,args=(nDropHDM,nExp,nMant,True,True)),singleEvent.calibCharge.apply(encode,args=(nDropLDM,nExp,nMant,True,True)))
    singleEvent['pass_135'] = singleEvent.calibCharge>=singleEvent.threshold_ADC
    
#    print (singleEvent)
#    print (singleEvent.index.values[0][1])
#    print (df.reset_index)[1]


    
    
    
    calibInputLines += "%i, "%BC
    algoInputLines += "%i, "%BC
    algoOutputLines += "%i, "%BC
    formattedDataLines += "%i, "%BC
    formattedDataLinesBestChoice += "%i, "%BC
    
    for x in singleEvent.decodedCharge.values:
        calibInputLines += "%i, "%x
    calibInputLines = calibInputLines[:-2]+"\n"

    for x in singleEvent.calibCharge.values:
        algoInputLines += "%i, "%x
    algoInputLines = algoInputLines[:-2]+"\n"
    
    NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2 = getAlgoBlockOutput(singleEvent.reset_index(['subdet','layer','wafer'],drop=True))
    ADD_MAP_INT = bitmapArrayToInt(ADD_MAP)[0]
    algoOutputLines += "%i, %i, "%(NTCQ, ADD_MAP_INT)
    for x in CHARGEQ:
        algoOutputLines += "%i, "%x
    for i in range(48-len(CHARGEQ)):
        algoOutputLines += "0, "
    algoOutputLines += "%i, %i, %i, %i \n"%( SUM, MOD_SUM_0, MOD_SUM_1, MOD_SUM_2)

    data = formatThresholdOutput(BC, NTCQ, ADD_MAP_INT, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2)
    formattedDataLines += data+" \n"
    
    
    
    ADD_MAP_BC ,CHARGES_BC ,SUM_BC = getAlgoBlockOutputBestChoice(singleEvent.reset_index(['subdet','layer','wafer'],drop=True), nTC_bestchoice)
    ADD_MAP_INT_BC = bitmapArrayToInt(ADD_MAP_BC)[0]
    algoOutputLinesBestChoice += "%i, "%BC
    for x in CHARGES_BC:
        algoOutputLinesBestChoice += "%i, "%x
    
    algoOutputLinesBestChoice += "%i, %i \n"%(ADD_MAP_INT_BC , SUM_BC)   
    
    data = formatBestChoiceOutput(BC, nTC_bestchoice , ADD_MAP_INT_BC, CHARGES_BC,SUM_BC)
    formattedDataLinesBestChoice += data+" \n"
    
    
print (algoOutputLinesBestChoice)    
print (formattedDataLinesBestChoice)   
#print (len(formattedDataLines))    
    
    
    

BC, CHARGE_0, CHARGE_1, CHARGE_2, ADD_MAP, SUM
0, 38, 34, 37, 13194139533824, 169 
1, 43, 41, 44, 416611827712, 176 
2, 50, 41, 56, 1099511889936, 184 
3, 48, 50, 45, 21990232571904, 186 
4, 49, 42, 44, 1099511627824, 175 
5, 39, 49, 39, 2199023260160, 175 
6, 45, 47, 45, 12352, 182 
7, 46, 52, 49, 17592186044496, 186 
8, 41, 41, 42, 1600, 178 
9, 40, 50, 42, 184549376, 176 
10, 49, 44, 46, 12885426176, 178 
11, 42, 41, 43, 8796160393216, 181 
12, 38, 40, 42, 4398046511233, 164 
13, 42, 40, 50, 137438953544, 178 
14, 45, 42, 50, 549755831296, 180 
15, 44, 52, 42, 140737488359425, 180 
16, 48, 51, 52, 17600775979040, 186 
17, 38, 40, 39, 70368744214528, 175 
18, 41, 44, 43, 82176, 179 
19, 40, 40, 40, 536873024, 171 
20, 51, 39, 42, 35184506306592, 177 
21, 40, 41, 41, 68757225472, 172 
22, 42, 40, 45, 429496729600, 176 
23, 47, 48, 45, 81, 181 
24, 41, 46, 43, 532736, 182 
25, 40, 41, 48, 140754668225024, 178 
26, 48, 36, 46, 8800387989760, 171 
27, 45, 42, 48, 154619084800, 183 
28, 3

In [22]:
with open('CSV_Files/FormattingBlockOutput_BestChoice.csv','w') as _file:
    _file.write(formattedDataLinesBestChoice)

FileNotFoundError: [Errno 2] No such file or directory: 'CSV_Files/FormattingBlockOutput_BestChoice.csv'

In [ ]:
with open('CSV_Files/AlgoBlockOutput_BestChoice.csv','w') as _file:
    _file.write(algoOutputLinesBestChoice)

In [ ]:
with open('CSV_Files/CalibBlockInput.csv','w') as _file:
    _file.write(calibInputLines)

In [ ]:
with open('CSV_Files/AlgoBlockInput.csv','w') as _file:
    _file.write(algoInputLines)

In [ ]:
with open('CSV_Files/AlgoBlockOutput_Threshold_135.csv','w') as _file:
    _file.write(algoOutputLines)

In [ ]:
with open('CSV_Files/FormattingBlockOutput_Threshold_135.csv','w') as _file:
    _file.write(formattedDataLines)

In [ ]:
with open('CSV_Files/Control_Module_Info.csv','w') as _file:
    values = "SUBDET, LAYER, MODULE\n"
    values += "%i, %i, %i"%(subdet, layer, wafer)
    values += "\n\n\n"
    values += str(eventTracker)
    _file.write(values)
    
with open('CSV_Files/Control_Threshold_Levels.csv','w') as _file:
    threshold_values = str(singleEvent.threshold_ADC.values.tolist())
    threshold_values = threshold_values[1:-1]
    line = ''
    for i in range(48):
        line += 'THRESHV_%i, '%i
    line = line[:-2] + '\n'
    line += threshold_values + '\n'
    _file.write(line)

    
with open('CSV_Files/Control_Calibration_Values.csv','w') as _file:
    calib_values = str(calibRegisters.tolist())
    calib_values = calib_values[1:-1]
    line = ''
    for i in range(48):
        line += 'CALV_%i, '%i
    line = line[:-2] + '\n'
    line += calib_values + '\n'
    _file.write(line)
